## Training

In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import namedtuple
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score,make_scorer
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
from collections import Counter ,OrderedDict
from itertools import compress
# TODO: Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [2]:
Dataset=namedtuple('Dataset','exchange df')
DatasetMLModel= namedtuple('DatasetMLModel','exchange df train_size tscv_split test_size')
Regressor= namedtuple('Regressor','name regressor_class params type')
FeatureSelection= namedtuple('FeatureSelection','dataset regressor params RFECV')

In [9]:
with open('datasets/set_of_datasets.pkl', 'rb') as input1:
    set_of_datasets = pickle.load(input1)
with open('datasets/fs_results.pkl', 'rb') as input2:
    fs_results = pickle.load(input2) 
with open('datasets/fs_results2.pkl', 'rb') as input3:
    fs_results2 = pickle.load(input3) 

In [4]:
regressors={
    'LinearRegression': Regressor('LinearRegression', LinearRegression, None,'linear_model'), 
    'Lasso':Regressor('Lasso', Lasso, None,'linear_model'),
#Error it is for categorical features    'LogisticRegression':Regressor('LogisticRegression', LogisticRegression, None,'linear_model'),
    'DecisionTreeRegressor':Regressor('DecisionTreeRegressor', DecisionTreeRegressor, None,'tree'),
    'GradientBoostingRegressor':Regressor('GradientBoostingRegressor', GradientBoostingRegressor, None,'ensemble')
}
regressors_plus={
#     'Ridge':Regressor('Ridge', Ridge, None,'linear_model'),
,
}
parameters_per_regressor={
    'LinearRegression':{},
    'Lasso': { 'alpha':[100,100,1000,10000]},
    'SVR': {'C': [1,10,100,1000,10000], 'gamma': [0.01, 0.1,1 ,10,100], 'kernel': ['rbf', 'sigmoid', 'poly'],'max_iter':[500]},
    'KernelRidge': { 'alpha':[1,10,100,100],'gamma': [0.01, 0.1,1 ,10,100],'kernel': ['rbf', 'sigmoid', 'poly']},
    'DecisionTreeRegressor': {'max_depth':range(1,15),'criterion':['mse','friedman_mse']},
    'GradientBoostingRegressor':{}
}



In [5]:

def make_X_Y(dataset, features=['close'],target=['close_Target']):
    X=dataset.df[features]
    y=dataset.df[target]
    return X,y

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true and predicted values based on the metric chosen. """
    

    score = r2_score(y_true,y_predict)
    
    # Return the score
    return score


In [6]:
def fit_all(regressors,parameters_per_regressor,fs_results,set_of_datasets):
    columns=['Currency Pair','Regressor','Target','Dataset','Score','Features','BestEstimator']
    data=[]
    scoring_fnc = make_scorer(performance_metric)
    for dataset_key, fs_scores, target in fs_results:
        set_data=set_of_datasets[dataset_key]
        for index, row in fs_scores.iterrows():
            (currency_pair,regressor_key)=index
            dataset=set_data[currency_pair]
            features=row['features']
            X , y = make_X_Y(dataset,features,target)
            reg=regressors[regressor_key].regressor_class()
            parameters=parameters_per_regressor[regressor_key]
            cvts=TimeSeriesSplit(n_splits=dataset.tscv_split)
            print(  currency_pair,
                    regressor_key,
                    target,
                    dataset_key)
            grid = GridSearchCV(reg, parameters,scoring_fnc, cv=cvts) 
            grid = grid.fit(X, y)
            data_row=[
                currency_pair,
                regressor_key,
                target,
                dataset_key,
                grid.best_score_,
                features,
                grid.best_estimator_
            ]
            data.append(data_row)
    result_cv=pd.DataFrame(data,columns=columns)
    result_cv.set_index(['Currency Pair','Regressor','Target','Dataset'],inplace=True)
    return result_cv

def fit_all2(regressors,parameters_per_regressor,fs_results,set_of_datasets):
    columns=['Currency Pair','Regressor','Target','Dataset','Score','Features','BestEstimator']
    data=[]
    scoring_fnc = make_scorer(performance_metric)
    for dataset_key, fs_scores, target in fs_results:
        set_data=set_of_datasets[dataset_key]
        for index, row in fs_scores.iterrows():
            (currency_pair,regressor_key)=index
            dataset=set_data[currency_pair]
            features=row['features']
            X , y = make_X_Y(dataset,features,target)
            reg=regressors[regressor_key].regressor_class()
            parameters=parameters_per_regressor[regressor_key]
            cvts=TimeSeriesSplit(n_splits=dataset.tscv_split)
            print(  currency_pair,
                    regressor_key,
                    target,
                    dataset_key)
            grid = GridSearchCV(reg, parameters,scoring_fnc, cv=cvts) 
            grid = grid.fit(X, y)
            data_row=[
                currency_pair,
                regressor_key,
                target,
                dataset_key,
                grid.best_score_,
                features,
                grid.best_estimator_
            ]
            data.append(data_row)
            for regressor_key, regressor  in regressors_plus.items():
                reg=regressor.regressor_class()
                parameters=parameters_per_regressor[regressor_key]
                print(  currency_pair,
                    regressor_key,
                    target,
                    dataset_key)
                grid = GridSearchCV(reg, parameters,scoring_fnc, cv=cvts)   
                grid = grid.fit(X, y)
                data_row=[
                    currency_pair,
                    regressor_key,
                    target,
                    dataset_key,
                    grid.best_score_,
                    features,
                    grid.best_estimator_
                ]
                data.append(data_row)
    result_cv=pd.DataFrame(data,columns=columns)
    result_cv.set_index(['Currency Pair','Regressor','Target','Dataset'],inplace=True)
    return result_cv


    


In [7]:
scores=fit_all(regressors,parameters_per_regressor,fs_results,set_of_datasets)

btc_brl LinearRegression close_Target rescaled
btc_clp LinearRegression close_Target rescaled
btc_brl GradientBoostingRegressor close_Target rescaled
eth_btc LinearRegression close_Target rescaled
btc_brl DecisionTreeRegressor close_Target rescaled
btc_pen LinearRegression close_Target rescaled
eth_pen LinearRegression close_Target rescaled
eth_clp LinearRegression close_Target rescaled
eth_btc GradientBoostingRegressor close_Target rescaled
eth_cop LinearRegression close_Target rescaled
btc_cop GradientBoostingRegressor close_Target rescaled
btc_cop DecisionTreeRegressor close_Target rescaled
eth_pen GradientBoostingRegressor close_Target rescaled
eth_clp DecisionTreeRegressor close_Target rescaled
btc_clp DecisionTreeRegressor close_Target rescaled
btc_clp GradientBoostingRegressor close_Target rescaled
eth_btc DecisionTreeRegressor close_Target rescaled
btc_pen Lasso close_Target rescaled
eth_cop GradientBoostingRegressor close_Target rescaled
eth_clp GradientBoostingRegressor close

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [10]:
scores2=fit_all2(regressors,parameters_per_regressor,fs_results2,set_of_datasets)

btc_brl LinearRegression close_Target rescaled
btc_brl SVR close_Target rescaled
btc_brl KernelRidge close_Target rescaled


/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


btc_clp LinearRegression close_Target rescaled
btc_clp SVR close_Target rescaled


/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  %

btc_clp KernelRidge close_Target rescaled


...

eth_cop KernelRidge log_return_Target log_divided


/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solvin

btc_brl LinearRegression log_return_Target log_divided


/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

###### scores.sort_values('Score',ascending=False)

In [ ]:
scores2.sort_values('Score',ascending=False)